<a href="https://colab.research.google.com/github/parapinich/tmc-traffic-dashboard/blob/main/PROJECT_UAS_RKUHAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **PROSES CONNECT G-DRIVE**

In [ ]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


### **PROSES IMPORT LIBRARY YANG DIBUTUHKAN DAN MEMBACA DATASET**

In [ ]:
!pip install nltk
!pip install Sastrawi
!pip install textblob
!pip install deep-translator
!pip install wordcloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import string
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
url = '/content/gdrive/MyDrive/RKUHAP_UAS.xlsx'
df = pd.read_excel(url)
print(df.head())

                       created_at  \
0  Fri Oct 31 13:21:17 +0000 2025   
1  Fri Oct 31 13:18:43 +0000 2025   
2  Thu Oct 30 07:20:54 +0000 2025   
3  Tue Oct 28 13:11:38 +0000 2025   
4  Tue Oct 28 13:08:49 +0000 2025   

                                           full_text  
0  Pasal-pasal yang berpotensi melanggar hak asas...  
1  Pasal-pasal yang berpotensi melanggar hak asas...  
2  Komisi III DPR RI : RKUHAP Perkuat Pemberantas...  
3  Kalangan masyarakat sipil merasa masukan-masuk...  
4  Kalangan masyarakat sipil merasa masukan-masuk...  


### **MENGHAPUS BARIS KOSONG**

In [ ]:
print(f"\nJumlah baris SEBELUM manghapus komentar kosong: {len(df)}")

df_clean = df.dropna(subset=['full_text'])

print(f"Jumlah baris SETELAH manghapus komentar kosong: {len(df_clean)}")
print(f"Jumlah baris yang dihapus: {len(df) - len(df_clean)}")

print("\nDataset setelah menghapus komentar kosong:")
print(df_clean.head(10))


Jumlah baris SEBELUM manghapus komentar kosong: 1013
Jumlah baris SETELAH manghapus komentar kosong: 1013
Jumlah baris yang dihapus: 0

Dataset setelah menghapus komentar kosong:
                       created_at  \
0  Fri Oct 31 13:21:17 +0000 2025   
1  Fri Oct 31 13:18:43 +0000 2025   
2  Thu Oct 30 07:20:54 +0000 2025   
3  Tue Oct 28 13:11:38 +0000 2025   
4  Tue Oct 28 13:08:49 +0000 2025   
5  Mon Oct 27 04:45:57 +0000 2025   
6  Sat Oct 18 13:26:45 +0000 2025   
7  Sat Oct 18 08:00:00 +0000 2025   
8  Sat Oct 18 05:00:01 +0000 2025   
9  Sat Oct 18 02:00:02 +0000 2025   

                                           full_text  
0  Pasal-pasal yang berpotensi melanggar hak asas...  
1  Pasal-pasal yang berpotensi melanggar hak asas...  
2  Komisi III DPR RI : RKUHAP Perkuat Pemberantas...  
3  Kalangan masyarakat sipil merasa masukan-masuk...  
4  Kalangan masyarakat sipil merasa masukan-masuk...  
5  @RobbyDwk ini jadi bahaya kalo RUU Polri sama ...  
6  Baca Berita Baru : Relev

### **MEMBUAT RANGE TANGGAL DATASET DAN MEMISAHKAN ANTARA TANGGAL & WAKTU**

In [ ]:
from datetime import datetime
# Pisahkan tanggal dan waktu untuk setiap baris dalam DataFrame
for index, row in df.iterrows():
    try:
        date_time_obj = datetime.strptime(row['created_at'], '%a %b %d %H:%M:%S %z %Y')
        df.at[index, 'tanggal'] = date_time_obj.strftime('%a %b %d')
        df.at[index, 'waktu'] = date_time_obj.strftime('%H:%M:%S')
    except ValueError:
        print(f"Done: data berhasil di bagi: {row['created_at']}")

# Menampilkan DataFrame dengan kolom tanggal dan waktu terpisah
df.head(5)

,created_at,full_text,tanggal,waktu
0,Fri Oct 31 13:21:17 +0000 2025,Pasal-pasal yang berpotensi melanggar hak asas...,Fri Oct 31,13:21:17
1,Fri Oct 31 13:18:43 +0000 2025,Pasal-pasal yang berpotensi melanggar hak asas...,Fri Oct 31,13:18:43
2,Thu Oct 30 07:20:54 +0000 2025,Komisi III DPR RI : RKUHAP Perkuat Pemberantas...,Thu Oct 30,07:20:54
3,Tue Oct 28 13:11:38 +0000 2025,Kalangan masyarakat sipil merasa masukan-masuk...,Tue Oct 28,13:11:38
4,Tue Oct 28 13:08:49 +0000 2025,Kalangan masyarakat sipil merasa masukan-masuk...,Tue Oct 28,13:08:49


In [ ]:
df_clean['created_at'] = pd.to_datetime(df_clean['created_at'])
start = df_clean['created_at'].min()
end = df_clean['created_at'].max()

print(f"=== RANGE TANGGAL ===")
print(f"Mulai   : {start.strftime('%d %B %Y')}")
print(f"Selesai : {end.strftime('%d %B %Y')}")
print(f"Range   : {(end - start).days} hari")

/tmp/ipython-input-4177339767.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean['created_at'] = pd.to_datetime(df_clean['created_at'])


=== RANGE TANGGAL ===
Mulai   : 10 July 2025
Selesai : 31 October 2025
Range   : 112 hari


### **PROSES CASE FOLDING PADA DATASET**

In [ ]:
df_clean['content_clean'] = df_clean['full_text'].str.lower().str.strip()

print("\nDataset setelah case folding:")
print(df_clean[['content_clean', 'full_text']].tail(10))


Dataset setelah case folding:
                                          content_clean  \
1003       belom rkuhap loh ini https://t.co/jicf8cttx4   
1004  dpr sindir balik pengkritik rkuhap: kita atau ...   
1005  https://t.co/m60cq8lwwf panitia kerja dpr sepa...   
1006  saya mendengar dari anggota drafter rkuhap pem...   
1007                          rkuhap lolos lagi ini mah   
1008  dpr dan pemerintah rampungkan 1.676 dim rkuhap...   
1009            setuku rkuhap ningning sebagai penyidik   
1010  kalo rkuhap disahkan buru-buru tanpa dengerin ...   
1011  darurat pembahasan rkuhap aturan yang bisa mem...   
1012  kabarnya bandung bondowoso minder ngeliat pemb...   

                                              full_text  
1003       Belom RKUHAP loh ini https://t.co/JiCf8CtTX4  
1004  DPR Sindir Balik Pengkritik RKUHAP: Kita Atau ...  
1005  https://t.co/M60CQ8LWwf Panitia Kerja DPR sepa...  
1006  Saya mendengar Dari Anggota Drafter RKUHAP Pem...  
1007                         

### **MENGHAPUS URLs, MENTIONS, HASHTAG**

In [ ]:
print(f"\n Menghapus URLs, Mentions, Hashtags")

df_clean['content_clean'] = df_clean['content_clean'].apply(
    lambda x: re.sub(r'http\S+|www\S+|https\S+', '', str(x))
)
df_clean['content_clean'] = df_clean['content_clean'].apply(
    lambda x: re.sub(r'[@#]\w+', '', str(x))
)

print("✅ URL, mentions, hashtags berhasil dihapus")

print("Sample hasil comprehensive cleaning:")
print(df_clean[['full_text', 'content_clean']].head(5))


 Menghapus URLs, Mentions, Hashtags
✅ URL, mentions, hashtags berhasil dihapus
Sample hasil comprehensive cleaning:
                                           full_text  \
0  Pasal-pasal yang berpotensi melanggar hak asas...   
1  Pasal-pasal yang berpotensi melanggar hak asas...   
2  Komisi III DPR RI : RKUHAP Perkuat Pemberantas...   
3  Kalangan masyarakat sipil merasa masukan-masuk...   
4  Kalangan masyarakat sipil merasa masukan-masuk...   

                                       content_clean  
0  pasal-pasal yang berpotensi melanggar hak asas...  
1  pasal-pasal yang berpotensi melanggar hak asas...  
2  komisi iii dpr ri : rkuhap perkuat pemberantas...  
3  kalangan masyarakat sipil merasa masukan-masuk...  
4  kalangan masyarakat sipil merasa masukan-masuk...  


### **MENGHAPUS DUPLIKASI PADA DATASET**

In [ ]:
print(f"\nJumlah baris SEBELUM menghapus duplikat: {len(df_clean['content_clean'])}")

df_clean = df_clean.drop_duplicates(
    subset=['content_clean'],
    keep='first'
)


print(f"Jumlah baris SETELAH menghapus duplikat: {len(df_clean)}")
print(f"Jumlah baris yang dihapus: {len(df) - len(df_clean)}")

print("\nDataset setelah menghapus duplikat:")
print(df_clean.head(10))


Jumlah baris SEBELUM menghapus duplikat: 1013
Jumlah baris SETELAH menghapus duplikat: 938
Jumlah baris yang dihapus: 75

Dataset setelah menghapus duplikat:
                  created_at  \
0  2025-10-31 13:21:17+00:00   
2  2025-10-30 07:20:54+00:00   
3  2025-10-28 13:11:38+00:00   
5  2025-10-27 04:45:57+00:00   
6  2025-10-18 13:26:45+00:00   
7  2025-10-18 08:00:00+00:00   
8  2025-10-18 05:00:01+00:00   
9  2025-10-18 02:00:02+00:00   
10 2025-10-17 15:23:25+00:00   
11 2025-10-17 14:00:00+00:00   

                                            full_text  \
0   Pasal-pasal yang berpotensi melanggar hak asas...   
2   Komisi III DPR RI : RKUHAP Perkuat Pemberantas...   
3   Kalangan masyarakat sipil merasa masukan-masuk...   
5   @RobbyDwk ini jadi bahaya kalo RUU Polri sama ...   
6   Baca Berita Baru : Relevansi SEMA 5/2021 terka...   
7   Wakil Menteri Hukum RI Edward O.S. Hiariej yak...   
8   Mendapatkan sejumlah masukan dan kritik dari a...   
9   Wamen Hukum RI Edward O.S. H

In [ ]:
df_clean = df_clean.reset_index(drop=True)

In [ ]:
print(df_clean)

                   created_at  \
0   2025-10-31 13:21:17+00:00   
1   2025-10-30 07:20:54+00:00   
2   2025-10-28 13:11:38+00:00   
3   2025-10-27 04:45:57+00:00   
4   2025-10-18 13:26:45+00:00   
..                        ...   
933 2025-07-10 16:05:04+00:00   
934 2025-07-10 15:45:50+00:00   
935 2025-07-10 15:44:16+00:00   
936 2025-07-10 15:06:53+00:00   
937 2025-07-10 14:59:42+00:00   

                                             full_text  \
0    Pasal-pasal yang berpotensi melanggar hak asas...   
1    Komisi III DPR RI : RKUHAP Perkuat Pemberantas...   
2    Kalangan masyarakat sipil merasa masukan-masuk...   
3    @RobbyDwk ini jadi bahaya kalo RUU Polri sama ...   
4    Baca Berita Baru : Relevansi SEMA 5/2021 terka...   
..                                                 ...   
933  DPR dan Pemerintah Rampungkan 1.676 DIM RKUHAP...   
934            Setuku RKUHAP ningning sebagai penyidik   
935  Kalo RKUHAP disahkan buru-buru tanpa dengerin ...   
936  Darurat Pembahasan

### **PROSES MENGHAPUS PUNCTUATION DAN NUMBERS**

In [ ]:
print(f"\n Menghapus punctuation dan numbers")

df_clean['content_clean'] = df_clean['content_clean'].apply(
    lambda x: re.sub(r'[^\w\s]', ' ', str(x))
)
df_clean['content_clean'] = df_clean['content_clean'].apply(
    lambda x: re.sub(r'\d+', '', str(x))
)

print("✅ Punctuatuion dan numbers berhasil dihapus")
print("Sample hasil hapus punctuation dan angka:")
print(df_clean[['full_text', 'content_clean']].head(5))


 Menghapus punctuation dan numbers
✅ Punctuatuion dan numbers berhasil dihapus
Sample hasil hapus punctuation dan angka:
                                           full_text  \
0  Pasal-pasal yang berpotensi melanggar hak asas...   
1  Komisi III DPR RI : RKUHAP Perkuat Pemberantas...   
2  Kalangan masyarakat sipil merasa masukan-masuk...   
3  @RobbyDwk ini jadi bahaya kalo RUU Polri sama ...   
4  Baca Berita Baru : Relevansi SEMA 5/2021 terka...   

                                       content_clean  
0  pasal pasal yang berpotensi melanggar hak asas...  
1  komisi iii dpr ri   rkuhap perkuat pemberantas...  
2  kalangan masyarakat sipil merasa masukan masuk...  
3   ini jadi bahaya kalo ruu polri sama rkuhap sa...  
4  baca berita baru   relevansi sema   terkait pr...  


### **PROSES MENGHAPUS WHITESPACE**

In [ ]:
print(f"\n Menghapus whitespace")
df_clean['content_clean'] = df_clean['content_clean'].apply(
    lambda x: re.sub(r'\s+', ' ', str(x)).strip()
)
print("✅ Whitespace dinormalisasi")
print("Sample hasil hapus whitespace:")
print(df_clean[['full_text', 'content_clean']].head(10))


 Menghapus whitespace
✅ Whitespace dinormalisasi
Sample hasil hapus whitespace:
                                           full_text  \
0  Pasal-pasal yang berpotensi melanggar hak asas...   
1  Komisi III DPR RI : RKUHAP Perkuat Pemberantas...   
2  Kalangan masyarakat sipil merasa masukan-masuk...   
3  @RobbyDwk ini jadi bahaya kalo RUU Polri sama ...   
4  Baca Berita Baru : Relevansi SEMA 5/2021 terka...   
5  Wakil Menteri Hukum RI Edward O.S. Hiariej yak...   
6  Mendapatkan sejumlah masukan dan kritik dari a...   
7  Wamen Hukum RI Edward O.S. Hiariej menjabarkan...   
8  d. mengganti biaya yang ditimbulkan dari akiba...   
9  Siniar Open Minded di channel YouTube Diskursu...   

                                       content_clean  
0  pasal pasal yang berpotensi melanggar hak asas...  
1  komisi iii dpr ri rkuhap perkuat pemberantasan...  
2  kalangan masyarakat sipil merasa masukan masuk...  
3  ini jadi bahaya kalo ruu polri sama rkuhap sam...  
4  baca berita baru relevan

### **HAPUS SUPERSCRIPT, SUBSCRIPT DAN ANGKA**

In [ ]:
def remove_superscript_subscript_numbers(text):
  # Blok ini hanya untuk menangani jika input BUKAN string (misal: sudah None atau angka)
  if not isinstance(text, str):
    return text

  # Logika pembersihan HARUS berada di luar blok IF
  # (Indentasi sejajar dengan 'if' atau lebih sedikit)

  # Hapus karakter superscript dan subscript unicode
  # (Saya juga perbaiki sedikit typo di regex \\uu2080 menjadi \\u2080)
  text = re.sub(r'[\u00B2\u00B3\u00B9\u2070\u207A-\u207F\u2080-\u2089\u208A-\u208F]', '', text)

  # Hapus angka biasa (ini sebenarnya sudah dilakukan di sel 13, tapi tidak masalah)
  text = re.sub(r'\d', '', text)

  # Hapus spasi berlebih akibat penghapusan karakter
  text = re.sub(r'\s+', ' ', text).strip()

  return text # Return ini adalah untuk teks yang SUDAH diproses

# Terapkan fungsi yang sudah benar
df_clean['content_clean'] = df_clean['content_clean'].apply(remove_superscript_subscript_numbers)
print(df_clean.head()) # Print head() saja agar tidak terlalu panjang

                 created_at  \
0 2025-10-31 13:21:17+00:00   
1 2025-10-30 07:20:54+00:00   
2 2025-10-28 13:11:38+00:00   
3 2025-10-27 04:45:57+00:00   
4 2025-10-18 13:26:45+00:00   

                                           full_text  \
0  Pasal-pasal yang berpotensi melanggar hak asas...   
1  Komisi III DPR RI : RKUHAP Perkuat Pemberantas...   
2  Kalangan masyarakat sipil merasa masukan-masuk...   
3  @RobbyDwk ini jadi bahaya kalo RUU Polri sama ...   
4  Baca Berita Baru : Relevansi SEMA 5/2021 terka...   

                                       content_clean  
0  pasal pasal yang berpotensi melanggar hak asas...  
1  komisi iii dpr ri rkuhap perkuat pemberantasan...  
2  kalangan masyarakat sipil merasa masukan masuk...  
3  ini jadi bahaya kalo ruu polri sama rkuhap sam...  
4  baca berita baru relevansi sema terkait praper...  


### **NORMALISASI SLANG WORD**

In [ ]:
import pandas as pd

# Fungsi penggantian kata tidak baku
def replace_taboo_words(text, kamus_tidak_baku):
    if isinstance(text, str):
        words = text.split()
        replaced_words = []
        kalimat_baku = []
        kata_diganti = []
        kata_tidak_baku_hash = []

        for word in words:
            if word in kamus_tidak_baku:
                baku_word = kamus_tidak_baku[word]
                if isinstance(baku_word, str) and all(char.isalpha() for char in baku_word):
                    replaced_words.append(baku_word)
                    kalimat_baku.append(baku_word)
                    kata_diganti.append(word)
                    kata_tidak_baku_hash.append(hash(word))
                # else:
                #     replaced_words.append('<karakter eksplisit>')
                #     kalimat_baku.append(word)
            else:
                replaced_words.append(word)
        replaced_text = ' '.join(replaced_words)
    else:
        replaced_text = ''
        kalimat_baku = []
        kata_diganti = []
        kata_tidak_baku_hash = []

    return replaced_text, kalimat_baku, kata_diganti, kata_tidak_baku_hash

In [ ]:
# Baca kamus kata tidak baku
kamus_data = pd.read_excel('/content/gdrive/MyDrive/kamuskatabaku.xlsx')
kamus_tidak_baku = dict(zip(kamus_data['tidak_baku'], kamus_data['kata_baku']))

In [ ]:
# Terapkan fungsi penggantian kata tidak baku
df_clean['normalisasi'], df_clean['Kata_Baku'], df_clean['Kata_Tidak_Baku'], df_clean['Kata_Tidak_Baku_Hash'] = zip(*df_clean['content_clean'].apply(lambda x: replace_taboo_words(x, kamus_tidak_baku)))
# df_clean.head(100)

# The following lines are problematic as 'data' is not defined and some columns do not exist in df_clean.
# df =  pd.DataFrame(data[['tanggal','waktu','username','quote_count','reply_count','retweet_count',
#                          'favorite_count','full_text','cleaning','case_folding','normalisasi']])

# Display the updated df_clean with the new normalization columns
df_clean.head(5)

,created_at,full_text,content_clean,normalisasi,Kata_Baku,Kata_Tidak_Baku,Kata_Tidak_Baku_Hash
0,2025-10-31 13:21:17+00:00,Pasal-pasal yang berpotensi melanggar hak asas...,pasal pasal yang berpotensi melanggar hak asas...,pasal pasal yang berpotensi melanggar hak asas...,[],[],[]
1,2025-10-30 07:20:54+00:00,Komisi III DPR RI : RKUHAP Perkuat Pemberantas...,komisi iii dpr ri rkuhap perkuat pemberantasan...,komisi iii dpr ri rkuhap perkuat pemberantasan...,[],[],[]
2,2025-10-28 13:11:38+00:00,Kalangan masyarakat sipil merasa masukan-masuk...,kalangan masyarakat sipil merasa masukan masuk...,kalangan masyarakat sipil merasa masukan masuk...,[],[],[]
3,2025-10-27 04:45:57+00:00,@RobbyDwk ini jadi bahaya kalo RUU Polri sama ...,ini jadi bahaya kalo ruu polri sama rkuhap sam...,ini jadi bahaya kalau ruu polri sama rkuhap sa...,"[kalau, sampai]","[kalo, sampe]","[-5049029360618978236, 2268791341160494570]"
4,2025-10-18 13:26:45+00:00,Baca Berita Baru : Relevansi SEMA 5/2021 terka...,baca berita baru relevansi sema terkait praper...,baca berita baru relevansi sema terkait praper...,"[di, sama]","[d, ma]","[8820729364758646341, 4610952446942376156]"


### **MELAKUKAN PROSES TOKENISASI**

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
df['token_split'] = df_clean['content_clean'].str.lower().str.split()
print("\nTokenisasi sederhana:")
print(df[['full_text', 'token_split']])


Tokenisasi sederhana:
                                              full_text  \
0     Pasal-pasal yang berpotensi melanggar hak asas...   
1     Pasal-pasal yang berpotensi melanggar hak asas...   
2     Komisi III DPR RI : RKUHAP Perkuat Pemberantas...   
3     Kalangan masyarakat sipil merasa masukan-masuk...   
4     Kalangan masyarakat sipil merasa masukan-masuk...   
...                                                 ...   
1008  DPR dan Pemerintah Rampungkan 1.676 DIM RKUHAP...   
1009            Setuku RKUHAP ningning sebagai penyidik   
1010  Kalo RKUHAP disahkan buru-buru tanpa dengerin ...   
1011  Darurat Pembahasan RKUHAP Aturan yang bisa mem...   
1012  Kabarnya Bandung Bondowoso minder ngeliat pemb...   

                                            token_split  
0     [pasal, pasal, yang, berpotensi, melanggar, ha...  
1     [komisi, iii, dpr, ri, rkuhap, perkuat, pember...  
2     [kalangan, masyarakat, sipil, merasa, masukan,...  
3     [ini, jadi, bahaya, kalo, ruu,

### **PROSES MENGHAPUS STOPWORDS**

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# ganti path sesuai lokasi file di Google Drive
stopwords_df = pd.read_excel('/content/gdrive/MyDrive/stopwordbahasa.xlsx')

# asumsikan kolom bernama 'stopword'
stopwords_list = stopwords_df['stopword'].dropna().tolist()

print("Jumlah stopwords:", len(stopwords_list))
print("Contoh stopwords:", stopwords_list[:20])


Jumlah stopwords: 758
Contoh stopwords: ['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara']


In [ ]:
def remove_indonesian_stopwords(text):
    # ubah teks jadi string lalu split
    words = str(text).split()
    # hapus stopwords berdasarkan list dari Excel
    filtered_words = [word for word in words if word.lower() not in stopwords_list]
    return ' '.join(filtered_words)

# terapkan ke dataframe
df_clean['content_clean'] = df_clean['content_clean'].apply(remove_indonesian_stopwords)

print("✅ Stopwords dari Excel berhasil digunakan")
print(df_clean[['content_clean', 'full_text']])

✅ Stopwords dari Excel berhasil digunakan
                                         content_clean  \
0    berpotensi melanggar hak asasi manusia dihilan...   
1    komisi iii dpr ri perkuat pemberantasan korups...   
2    kalangan masyarakat sipil masukan masukan draf...   
3                    bahaya kalo ruu polri sampe lolos   
4    berita relevansi sema terkait praperadilan aya...   
..                                                 ...   
931  mendengar anggota drafter pemerintah bingung d...   
933                      dpr pemerintah rampungkan dim   
935  kalo disahkan buru buru dengerin suara publik ...   
936  darurat pembahasan aturan memaksa penangkapan ...   
937  kabarnya bandung bondowoso minder ngeliat pemb...   

                                             full_text  
0    Pasal-pasal yang berpotensi melanggar hak asas...  
1    Komisi III DPR RI : RKUHAP Perkuat Pemberantas...  
2    Kalangan masyarakat sipil merasa masukan-masuk...  
3    @RobbyDwk ini jadi bahaya ka

### **PROSES STEMMING DATASET**

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming_indonesian_text(text):
  if not isinstance(text, str):
    return text
  words = text.split()
  stemmed_words = [stemmer.stem(word) for word in words]
  return ' '.join(stemmed_words)

print("Melakukan stemming Bahasa Indonesia. . .")
df_clean['content_stem'] = df_clean['content_clean'].apply(stemming_indonesian_text)

print("✅ Stemming selesai!\n")
print(df_clean[['content_clean', 'content_stem']].head())

Melakukan stemming Bahasa Indonesia. . .
✅ Stemming selesai!

                                       content_clean  \
0  berpotensi melanggar hak asasi manusia dihilan...   
1  komisi iii dpr ri perkuat pemberantasan korups...   
2  kalangan masyarakat sipil masukan masukan draf...   
3                  bahaya kalo ruu polri sampe lolos   
4  berita relevansi sema terkait praperadilan aya...   

                                        content_stem  
0  potensi langgar hak asasi manusia hilang draf ...  
1        komisi iii dpr ri kuat berantas korupsi tvr  
2  kalang masyarakat sipil masuk masuk draf perin...  
3                  bahaya kalo ruu polri sampe lolos  
4  berita relevansi sema kait praperadilan ayat h...  


### VADER

In [ ]:
from deep_translator import GoogleTranslator

def to_english(text: str) -> str:
  s = str(text)
  if not s.strip():
    return s
  try:
    # Using GoogleTranslator from deep_translator
    return GoogleTranslator(source='id', target='en').translate(text=s)
  except Exception as e:
    print(f"[WARN] Translation failed for text: {s[:50]}... Error: {e}")
    return s

df_clean['content_en'] = df_clean['content_clean'].apply(to_english)

# Cek Hasil
print(df_clean[['content_clean', 'content_en']].head(10))

                                       content_clean  \
0  berpotensi melanggar hak asasi manusia dihilan...   
1  komisi iii dpr ri perkuat pemberantasan korups...   
2  kalangan masyarakat sipil masukan masukan draf...   
3                  bahaya kalo ruu polri sampe lolos   
4  berita relevansi sema terkait praperadilan aya...   
5  wakil menteri hukum ri edward o s hiariej draf...   
6  masukan kritik akademisi wamenkum ri prof eddy...   
7  wamen hukum ri edward o s hiariej menjabarkan ...   
8  d mengganti biaya ditimbulkan akibat tindak pi...   
9  siniar open minded channel youtube diskursus n...   

                                          content_en  
0  potentially violating human rights was removed...  
1  Commission III of the Indonesian House of Repr...  
2  civil society input on the government's draft ...  
3           It's dangerous if the police bill passes  
4  sema relevance news related to pretrial paragr...  
5  Deputy Minister of Law of the Republic of Indo... 

In [ ]:
!pip install vaderSentiment

In [ ]:
df_clean = df_clean[df_clean['content_en'].str.split().str.len() > 5]

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def get_vader_score_label(text_en):
    scores = analyzer.polarity_scores(str(text_en))
    compound = scores['compound']

    if compound >= 0.05:
        return compound, 'positive'
    elif compound <= -0.05:
        return compound, 'negative'
    else:
        return compound, 'neutral'

print("Melabeli dengan VADER (2 kelas preparation)...")

df_clean[['compound', 'vader_label']] = df_clean['content_en'].apply(
    lambda x: pd.Series(get_vader_score_label(x))
)

print(df_clean['vader_label'].value_counts())


Melabeli dengan VADER (2 kelas preparation)...
vader_label
negative    412
positive    313
neutral     106
Name: count, dtype: int64


In [ ]:
df_binary = df_clean[df_clean['vader_label']
!= 'neutral'].copy()

print(df_binary['vader_label'].value_counts())


vader_label
negative    412
positive    313
Name: count, dtype: int64


### TAHAP EKSTRAKSI FITUR (TF-IDF)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
)


In [ ]:
X = df_binary['content_en']
y = df_binary['vader_label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [ ]:
tfidf = TfidfVectorizer(
    ngram_range=(1,2),
    max_features=7000,
    min_df=2,
    max_df=0.9,
    sublinear_tf=True
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [ ]:
model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced'
)

model.fit(X_train_tfidf, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000)

In [ ]:
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Accuracy: 0.7655172413793103

Classification Report:
              precision    recall  f1-score   support

    negative       0.78      0.82      0.80        82
    positive       0.75      0.70      0.72        63

    accuracy                           0.77       145
   macro avg       0.76      0.76      0.76       145
weighted avg       0.76      0.77      0.76       145

